In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import kgof
import kgof.data as data
import kgof.density as density
import kgof.goftest as gof
import kgof.kernel as kernel
import kgof.util as util
import matplotlib
import matplotlib.pyplot as plt
import autograd.numpy as np
import scipy.stats as stats
import tensorflow as tf

In [ ]:
# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 18
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## Shapiro-Wilk

In [ ]:
# Can it work for the multivariate case?
#x = np.random.randn(200, 2).dot(np.random.rand(2, 2)) + 3
x = stats.t.rvs(df=5, size=500) 
st, pval = stats.shapiro(x)
print 'stat: {0}, pval: {0}'.format( st, pval)

## Anderson-Darling

In [ ]:
result =  stats.anderson(x, dist='norm')
result

In [ ]:
A = np.reshape(range(12), (4, 3))
A

In [ ]:
A*np.random.randn(3)

## Hashing function handles

In [ ]:
f1 = lambda x: x**2
hash(f1.__code__)

In [ ]:
class Func(object):
    def __call__(self, x):
        return x**3

f2 = Func()
f2(2)

In [ ]:
hash(f2)

In [ ]:
import pickle
iso = density.IsotropicNormal(np.zeros(2), 1)

## Test autograd

In [ ]:
import autograd as ag
#import numpy as np
import autograd.numpy as np

def kgauss(X, Y, sigma2=1):
    sumx = np.sum(X**2, 1)
    sumy = np.sum(Y**2, 1)
    D2 = sumx.reshape(-1, 1) - 2*X.dot(Y.T) + sumy.reshape(1, -1)
    K = np.exp(-D2/(2.0*sigma2))
    return K

def exp(X):
    Y = X
    d = {'a': Y}
    return np.exp(-d['a'])[0, :]

In [ ]:
n = 3
d = 2
X = np.random.randn(n, d)

#K = kgauss(X, X)
#g_kg = ag.grad(kgauss)
#g_kg(X, X)

E = exp(X)
g_exp = ag.elementwise_grad(exp)
j_exp = ag.jacobian(exp)

In [ ]:
E

In [ ]:
g_exp(X)

In [ ]:
j_exp(X)

## Test autograd with instance variables

In [ ]:
import autograd
import autograd.numpy as np
import copy

In [ ]:
seed = 8
mean = np.zeros(d)
variance = 1
isonorm = density.IsotropicNormal(mean, variance)

# only one dimension of the mean is shifted
#draw_mean = mean + np.hstack((1, np.zeros(d-1)))
draw_mean = mean +0
draw_variance = variance + 1
X = util.randn(n, d, seed=seed)*np.sqrt(draw_variance) + draw_mean
dat = data.Data(X)


In [ ]:
def optimize_obj(p, data, k_cls, k_params, V0):
    k = k_cls.from_params(**k_params)
    fssd = gof.FSSD(p, k, V0, seed=seed+1)
    fea_tensor = fssd.feature_tensor(X)
    u_mean, u_variance = gof.FSSD.ustat_h1_mean_variance(fea_tensor, return_variance=True)
    # mean/sd criterion 
    obj = u_mean/np.sqrt(u_variance)
    return obj

In [ ]:
J = 3
sig2 = util.meddistance(X, subsample=1000)**2
k_params = {'sigma2': sig2}
#k = kernel.KGauss(sig2)

# random test locations
V = util.fit_gaussian_draw(X, J, seed=seed+1)


## Autograd + scipy.optimize

In [ ]:
def f(X):
    n, d = X.shape
    mean = np.hstack((2, np.zeros(d-1)))
    norm2 = np.sum((X-mean)**2, 1)
    return norm2

g = autograd.elementwise_grad(f)

In [ ]:
with util.NumpySeedContext(seed=2):
    X = np.random.randint(1, 4+1, 6).reshape(3, 2)*1.0
X

In [ ]:
f(X)

In [ ]:
g(X)

In [ ]:
import scipy

n = 3
d = 2

obj = lambda X: np.sum(f(X))
flat_obj = lambda x: obj(np.reshape(x, (n, d)))
g_obj = autograd.elementwise_grad(flat_obj)

X0 = np.random.randn(n, d)*20 + 3
x0 = np.reshape(X0, -1)
opt_result = scipy.optimize.minimize(flat_obj, x0, method='BFGS', jac=g_obj)
X_opt = opt_result['x'].reshape(n, d)
X_opt

## Autograd + scipy.optimize for FSSD

In [ ]:
J = 1
seed = 8
n = 10
d = 1
mean = np.zeros(d)
variance = 1
p = density.IsotropicNormal(mean, variance)


# only one dimension of the mean is shifted
#draw_mean = mean + np.hstack((1, np.zeros(d-1)))
draw_mean = mean +1
draw_variance = variance + 0
X = util.randn(n, d, seed=seed)*np.sqrt(draw_variance) + draw_mean
dat = data.Data(X)

sig2 = util.meddistance(X, subsample=1000)**2
k_params = {'sigma2': sig2}
#k = kernel.KGauss(sig2)

# random test locations
V0 = util.fit_gaussian_draw(X, J, seed=seed+1)
tr, te = dat.split_tr_te(tr_proportion=0.5, seed=4)

In [ ]:
def obj(sqrt_gwidth, V):
    return -gof.GaussFSSD.power_criterion(p, tr, sqrt_gwidth**2, V)

flatten = lambda gwidth, V: np.hstack((gwidth, V.reshape(-1)))

def unflatten(x):
    sqrt_gwidth = x[0]
    V = np.reshape(x[1:], (J, d))
    return sqrt_gwidth, V

def flat_obj(x):
    sqrt_gwidth, V = unflatten(x)
    return obj(sqrt_gwidth, V)

g_obj = autograd.elementwise_grad(flat_obj)
x0 = flatten(np.sqrt(sig2), V0)

In [ ]:
import autograd.util
autograd.util.quick_grad_check(flat_obj, x0)

In [ ]:
g_obj(x0)

In [ ]:
opt_result = scipy.optimize.minimize(
    flat_obj, x0, method='BFGS', 
    #jac=g_obj
)
x_opt = opt_result['x']
sq_gw_opt, V_opt = unflatten(x_opt)
gw_opt = sq_gw_opt**2

In [ ]:
opt_result

In [ ]:
gw_opt

In [ ]:
V_opt

In [ ]:
V0

## Stacking

In [ ]:
X = np.reshape(range(4*3*2), (4, 3, 2))

In [ ]:
X

In [ ]:
np.transpose(X, (1, 2, 0))

In [ ]:
np.concatenate([mat[:, np.newaxis] for mat in X], axis=2)